In [ ]:
import warnings
import logging
from ctransformers import AutoConfig, AutoModelForCausalLM, Config

logging.basicConfig(level=logging.ERROR)


warnings.filterwarnings('ignore')

### Dowload Mistral

In [ ]:
!model=models/mistral-7b-instruct-v0.1.Q4_K_M.gguf

# !if [[ ! -f $model ]]; then wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf -O models/mistral-7b-instruct-v0.1.Q4_K_M.gguf; fi

### Load the llmodel 

In [ ]:
conf = AutoConfig(Config(temperature=0.8, repetition_penalty=1.2, max_new_tokens=1024, context_length=2048))

llm = AutoModelForCausalLM.from_pretrained("models/mistral-7b-instruct-v0.1.Q4_K_M.gguf", model_type="mistral", config=conf)

## Create Chat 

In [ ]:
import torch 

# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
model_name_or_path = "mistralai/Mistral-7B-Instruct-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

model_inputs = encodeds.to(device)
model.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])